# User2Vec
Based on: https://ieeexplore.ieee.org/document/8875952/

How it works:
- Run doc2vec
- Average vector representations for each user


In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import eland as ed
from sklearn.manifold import TSNE

from src.models import User2Vec, tokenize

# uncomment this if database is not already open (and give ES a couple minutes to set up)
#!make database

ed_df = ed.read_es('localhost', 'twitter')

/Users/jfonseca/miniconda3/envs/solve-iwmi/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
2020-08-29 01:12:28,739 - elasticsearch - INFO - GET http://localhost:9200/twitter/_mapping [status:200 request:0.213s]
2020-08-29 01:12:28,844 - elasticsearch - INFO - GET http://localhost:9200/twitter/_field_caps?fields=%2A [status:200 request:0.102s]


In [2]:
df = ed_df[[
    'tweet_id', 'original_tweet_id_str', 
    'user_id', 'name', 'full_text_processed', 
    'sentiment', 'followers_count'
]].to_pandas().fillna(np.nan)

2020-08-29 01:12:29,404 - elasticsearch - INFO - POST http://localhost:9200/twitter/_search?scroll=5m&size=1000 [status:200 request:0.318s]
2020-08-29 01:12:29,646 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.210s]
2020-08-29 01:12:29,907 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.233s]
2020-08-29 01:12:30,289 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.344s]
2020-08-29 01:12:30,661 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.353s]
2020-08-29 01:12:31,047 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.362s]
2020-08-29 01:12:31,350 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.196s]
2020-08-29 01:12:31,598 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.179s]
2020-08-29 01:12:31,754 - e

2020-08-29 01:12:47,131 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.408s]
2020-08-29 01:12:47,339 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.162s]
2020-08-29 01:12:47,785 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.404s]
2020-08-29 01:12:48,049 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.213s]
2020-08-29 01:12:48,255 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.164s]
2020-08-29 01:12:48,417 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.134s]
2020-08-29 01:12:48,602 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.166s]
2020-08-29 01:12:48,986 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.349s]
2020-08-29 01:12:49,226 - elasticsearch - INFO -

2020-08-29 01:13:37,567 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.250s]
2020-08-29 01:13:38,020 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.411s]
2020-08-29 01:13:38,303 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.259s]
2020-08-29 01:13:38,606 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.283s]
2020-08-29 01:13:38,829 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.185s]
2020-08-29 01:13:39,043 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.194s]
2020-08-29 01:13:39,471 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.407s]
2020-08-29 01:13:39,813 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.316s]
2020-08-29 01:13:39,988 - elasticsearch - INFO -

2020-08-29 01:13:55,148 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.113s]
2020-08-29 01:13:55,270 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.098s]
2020-08-29 01:13:55,403 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.108s]
2020-08-29 01:13:55,522 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.096s]
2020-08-29 01:13:55,669 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.113s]
2020-08-29 01:13:55,794 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.104s]
2020-08-29 01:13:55,920 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.107s]
2020-08-29 01:13:56,036 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.099s]
2020-08-29 01:13:56,209 - elasticsearch - INFO -

2020-08-29 01:14:11,947 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.102s]
2020-08-29 01:14:12,071 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.098s]
2020-08-29 01:14:12,276 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.178s]
2020-08-29 01:14:12,437 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.130s]
2020-08-29 01:14:12,622 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.162s]
2020-08-29 01:14:12,795 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.144s]
2020-08-29 01:14:12,998 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.178s]
2020-08-29 01:14:13,325 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.292s]
2020-08-29 01:14:14,015 - elasticsearch - INFO -

2020-08-29 01:14:30,603 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.249s]
2020-08-29 01:14:30,995 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.363s]
2020-08-29 01:14:31,256 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.234s]
2020-08-29 01:14:32,148 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.144s]
2020-08-29 01:14:32,270 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.103s]
2020-08-29 01:14:32,400 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.106s]
2020-08-29 01:14:32,588 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.165s]
2020-08-29 01:14:32,739 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.114s]
2020-08-29 01:14:32,883 - elasticsearch - INFO -

2020-08-29 01:15:04,712 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.107s]
2020-08-29 01:15:04,833 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.095s]
2020-08-29 01:15:05,022 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.169s]
2020-08-29 01:15:05,153 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.113s]
2020-08-29 01:15:05,300 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.122s]
2020-08-29 01:15:05,428 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.107s]
2020-08-29 01:15:05,541 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.091s]
2020-08-29 01:15:05,700 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.135s]
2020-08-29 01:15:06,586 - elasticsearch - INFO -

In [3]:
unique_docs = df['full_text_processed'].unique()
train_corpus = [tokenize(doc, tag) for doc, tag in zip(unique_docs, range(unique_docs.shape[0]))]

In [4]:
model = User2Vec(vector_size=10, min_count=2, epochs=500, random_state=0, workers=4)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-08-29 01:15:21,208 - gensim.models.base_any2vec - WARNING - consider setting layer size to a multiple of 4 for greater performance
2020-08-29 01:15:21,219 - gensim.models.doc2vec - INFO - collecting all words and their counts
2020-08-29 01:15:21,220 - gensim.models.doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-08-29 01:15:21,288 - gensim.models.doc2vec - INFO - PROGRESS: at example #10000, processed 153184 words (2295526/s), 15120 word types, 10000 tags
2020-08-29 01:15:21,384 - gensim.models.doc2vec - INFO - PROGRESS: at example #20000, processed 305222 words (1606763/s), 26151 word types, 20000 tags
2020-08-29 01:15:21,481 - gensim.models.doc2vec - INFO - PROGRESS: at example #30000, processed 460903 words (1697981/s), 34588 word types, 30000 tags
2020-08-29 01:15:21,559 - gensim.models.doc2vec - INFO - PROGRESS: at example #40000, processed 613179 words (1993534/s), 40926 word types, 40000 tags
2020-08-29 01:15:21,618 - gensim.mode

2020-08-29 01:16:17,806 - gensim.models.base_any2vec - INFO - EPOCH 4 - PROGRESS: at 40.42% examples, 157942 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:18,828 - gensim.models.base_any2vec - INFO - EPOCH 4 - PROGRESS: at 54.12% examples, 160297 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:19,889 - gensim.models.base_any2vec - INFO - EPOCH 4 - PROGRESS: at 67.65% examples, 158814 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:20,908 - gensim.models.base_any2vec - INFO - EPOCH 4 - PROGRESS: at 81.50% examples, 161751 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:21,950 - gensim.models.base_any2vec - INFO - EPOCH 4 - PROGRESS: at 95.80% examples, 162276 words/s, in_qsize 6, out_qsize 0
2020-08-29 01:16:22,100 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 01:16:22,149 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 01:16:22,173 - gensim.models.base_any2v

2020-08-29 01:16:55,732 - gensim.models.base_any2vec - INFO - EPOCH 9 - PROGRESS: at 40.42% examples, 163781 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:56,775 - gensim.models.base_any2vec - INFO - EPOCH 9 - PROGRESS: at 54.12% examples, 163924 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:57,837 - gensim.models.base_any2vec - INFO - EPOCH 9 - PROGRESS: at 67.65% examples, 161621 words/s, in_qsize 8, out_qsize 0
2020-08-29 01:16:58,842 - gensim.models.base_any2vec - INFO - EPOCH 9 - PROGRESS: at 79.51% examples, 160075 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:16:59,912 - gensim.models.base_any2vec - INFO - EPOCH 9 - PROGRESS: at 93.23% examples, 159038 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:17:00,250 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 01:17:00,320 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 01:17:00,331 - gensim.models.base_any2v

2020-08-29 01:17:33,998 - gensim.models.base_any2vec - INFO - EPOCH 14 - PROGRESS: at 40.42% examples, 162162 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:17:35,018 - gensim.models.base_any2vec - INFO - EPOCH 14 - PROGRESS: at 54.12% examples, 163600 words/s, in_qsize 8, out_qsize 0
2020-08-29 01:17:36,040 - gensim.models.base_any2vec - INFO - EPOCH 14 - PROGRESS: at 67.65% examples, 162596 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:17:37,109 - gensim.models.base_any2vec - INFO - EPOCH 14 - PROGRESS: at 81.50% examples, 163642 words/s, in_qsize 8, out_qsize 0
2020-08-29 01:17:38,147 - gensim.models.base_any2vec - INFO - EPOCH 14 - PROGRESS: at 95.80% examples, 163993 words/s, in_qsize 6, out_qsize 0
2020-08-29 01:17:38,309 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 01:17:38,332 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 01:17:38,384 - gensim.models.base_

2020-08-29 01:18:11,726 - gensim.models.base_any2vec - INFO - EPOCH 19 - PROGRESS: at 40.42% examples, 161789 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:18:12,730 - gensim.models.base_any2vec - INFO - EPOCH 19 - PROGRESS: at 54.12% examples, 163932 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:18:13,766 - gensim.models.base_any2vec - INFO - EPOCH 19 - PROGRESS: at 67.65% examples, 162475 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:18:14,846 - gensim.models.base_any2vec - INFO - EPOCH 19 - PROGRESS: at 81.50% examples, 163235 words/s, in_qsize 7, out_qsize 0
2020-08-29 01:18:15,863 - gensim.models.base_any2vec - INFO - EPOCH 19 - PROGRESS: at 95.80% examples, 164120 words/s, in_qsize 6, out_qsize 0
2020-08-29 01:18:16,015 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 01:18:16,050 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 01:18:16,096 - gensim.models.base_

...

2020-08-29 02:22:02,324 - gensim.models.base_any2vec - INFO - EPOCH 478 - PROGRESS: at 38.30% examples, 153746 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:03,376 - gensim.models.base_any2vec - INFO - EPOCH 478 - PROGRESS: at 51.86% examples, 156098 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:04,400 - gensim.models.base_any2vec - INFO - EPOCH 478 - PROGRESS: at 64.89% examples, 154805 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:05,407 - gensim.models.base_any2vec - INFO - EPOCH 478 - PROGRESS: at 77.52% examples, 155709 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:22:06,512 - gensim.models.base_any2vec - INFO - EPOCH 478 - PROGRESS: at 91.05% examples, 154593 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:06,960 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 02:22:07,003 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 02:22:07,033 - gensim.models.

2020-08-29 02:22:41,325 - gensim.models.base_any2vec - INFO - EPOCH 483 - PROGRESS: at 36.85% examples, 154070 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:42,388 - gensim.models.base_any2vec - INFO - EPOCH 483 - PROGRESS: at 48.75% examples, 151468 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:43,426 - gensim.models.base_any2vec - INFO - EPOCH 483 - PROGRESS: at 62.41% examples, 152428 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:22:44,566 - gensim.models.base_any2vec - INFO - EPOCH 483 - PROGRESS: at 76.86% examples, 153286 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:22:45,584 - gensim.models.base_any2vec - INFO - EPOCH 483 - PROGRESS: at 90.25% examples, 154385 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:22:46,174 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 02:22:46,229 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 02:22:46,252 - gensim.models.

2020-08-29 02:23:20,875 - gensim.models.base_any2vec - INFO - EPOCH 488 - PROGRESS: at 40.42% examples, 155865 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:23:21,922 - gensim.models.base_any2vec - INFO - EPOCH 488 - PROGRESS: at 53.28% examples, 155749 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:23:22,992 - gensim.models.base_any2vec - INFO - EPOCH 488 - PROGRESS: at 66.86% examples, 154919 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:23:24,003 - gensim.models.base_any2vec - INFO - EPOCH 488 - PROGRESS: at 80.22% examples, 157138 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:23:25,090 - gensim.models.base_any2vec - INFO - EPOCH 488 - PROGRESS: at 93.80% examples, 156154 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:23:25,368 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 02:23:25,403 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 02:23:25,442 - gensim.models.

2020-08-29 02:24:00,131 - gensim.models.base_any2vec - INFO - EPOCH 493 - PROGRESS: at 40.42% examples, 155948 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:24:01,157 - gensim.models.base_any2vec - INFO - EPOCH 493 - PROGRESS: at 54.12% examples, 158608 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:24:02,197 - gensim.models.base_any2vec - INFO - EPOCH 493 - PROGRESS: at 66.86% examples, 156432 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:24:03,214 - gensim.models.base_any2vec - INFO - EPOCH 493 - PROGRESS: at 79.55% examples, 156828 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:24:04,339 - gensim.models.base_any2vec - INFO - EPOCH 493 - PROGRESS: at 93.80% examples, 156334 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:24:04,594 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 02:24:04,633 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 02:24:04,683 - gensim.models.

2020-08-29 02:24:38,993 - gensim.models.base_any2vec - INFO - EPOCH 498 - PROGRESS: at 38.30% examples, 153711 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:24:40,040 - gensim.models.base_any2vec - INFO - EPOCH 498 - PROGRESS: at 51.19% examples, 153997 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:24:41,092 - gensim.models.base_any2vec - INFO - EPOCH 498 - PROGRESS: at 64.89% examples, 154059 words/s, in_qsize 8, out_qsize 0
2020-08-29 02:24:42,186 - gensim.models.base_any2vec - INFO - EPOCH 498 - PROGRESS: at 78.83% examples, 155785 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:24:43,188 - gensim.models.base_any2vec - INFO - EPOCH 498 - PROGRESS: at 92.60% examples, 156842 words/s, in_qsize 7, out_qsize 0
2020-08-29 02:24:43,606 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2020-08-29 02:24:43,646 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2020-08-29 02:24:43,681 - gensim.models.

In [5]:
users_id, user_vectors = model.infer_user_vectors(
    users = df['user_id'], 
    doc_words = df['full_text_processed']
)

vec_cols = [f'vec_{i}' for i in range(10)]
df_user_vecs = pd.DataFrame(user_vectors, columns=vec_cols)
df_user_vecs['user_id'] = users_id

100%|██████████| 474419/474419 [3:15:16<00:00, 40.49it/s]  


In [6]:
tsne = TSNE(random_state=0, n_jobs=-1, verbose=1)
user_embeddings = tsne.fit_transform(user_vectors)

df_user_vecs[['tsne_0','tsne_1']] = user_embeddings

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 194183 samples in 0.422s...
[t-SNE] Computed neighbors for 194183 samples in 142.086s...
[t-SNE] Computed conditional probabilities for sample 1000 / 194183
[t-SNE] Computed conditional probabilities for sample 2000 / 194183
[t-SNE] Computed conditional probabilities for sample 3000 / 194183
[t-SNE] Computed conditional probabilities for sample 4000 / 194183
[t-SNE] Computed conditional probabilities for sample 5000 / 194183
[t-SNE] Computed conditional probabilities for sample 6000 / 194183
[t-SNE] Computed conditional probabilities for sample 7000 / 194183
[t-SNE] Computed conditional probabilities for sample 8000 / 194183
[t-SNE] Computed conditional probabilities for sample 9000 / 194183
[t-SNE] Computed conditional probabilities for sample 10000 / 194183
[t-SNE] Computed conditional probabilities for sample 11000 / 194183
[t-SNE] Computed conditional probabilities for sample 12000 / 194183
[t-SNE] Computed conditional proba

[t-SNE] Computed conditional probabilities for sample 124000 / 194183
[t-SNE] Computed conditional probabilities for sample 125000 / 194183
[t-SNE] Computed conditional probabilities for sample 126000 / 194183
[t-SNE] Computed conditional probabilities for sample 127000 / 194183
[t-SNE] Computed conditional probabilities for sample 128000 / 194183
[t-SNE] Computed conditional probabilities for sample 129000 / 194183
[t-SNE] Computed conditional probabilities for sample 130000 / 194183
[t-SNE] Computed conditional probabilities for sample 131000 / 194183
[t-SNE] Computed conditional probabilities for sample 132000 / 194183
[t-SNE] Computed conditional probabilities for sample 133000 / 194183
[t-SNE] Computed conditional probabilities for sample 134000 / 194183
[t-SNE] Computed conditional probabilities for sample 135000 / 194183
[t-SNE] Computed conditional probabilities for sample 136000 / 194183
[t-SNE] Computed conditional probabilities for sample 137000 / 194183
[t-SNE] Computed con

In [7]:
df_meta = df.groupby('user_id').agg(
    {'full_text_processed': 'count', 'sentiment': 'mean', 'followers_count': 'max'}
).rename(columns={
        'full_text_processed': 'Tweet Count', 
        'sentiment':'Mean Sentiment', 
        'followers_count': 'Followers'
    }
)

df_user_vecs = df_user_vecs.set_index('user_id').join(df_meta)

In [8]:
df_user_vecs.to_csv('../data/results/iwmi_user2vec.csv')

In [9]:
tweet_vectors = np.array(df['full_text_processed'].apply(
    lambda doc: model.infer_vector(tokenize(doc))
).tolist())

vec_cols = [f'vec_{i}' for i in range(10)]
df_tweet_vecs = pd.DataFrame(tweet_vectors, columns=vec_cols, index=df['tweet_id'])

In [11]:
df_tweets = df.set_index('tweet_id').join(df_tweet_vecs)
df_tweets.to_csv('../data/results/iwmi_tweet2vec.csv')